# **Set according to environment (e.g. local, Google Colab...)**

In [17]:
project_folder = ''

# **Body**

In [18]:
from custom_libraries.image_dataset import *
from custom_libraries.miscellaneous import *
import numpy as np

In [19]:
def create_model(num_units):
    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.Dense(units=num_units, activation=None, kernel_initializer=tf.keras.initializers.HeNormal))
    model.add(tf.keras.layers.LeakyReLU(alpha=.01))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE),
                  metrics=['binary_crossentropy', 'acc'])

    return model

In [20]:
bs = 256
trials = 10
epochs = 2000
trees_set = [1, 32]

# classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)
classes = [[3, 5, 'usps']]
# TODO: verificare l'effetto di patience sul risultato di svhn (rischio underfitting)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60, mode='min')

history = np.zeros((len(classes), trials, len(trees_set), 2))
#history = np.load(project_folder+'results/fcnn_history.npy', allow_pickle=True)

for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    #if history[j, 0, 0, 0] != 0:
    #    continue

    test_ds = ImageDataset(ds, 'test', data_dir=None, USPS_dir=project_folder + 'USPS/')
    train_ds = ImageDataset(ds, 'train', data_dir=None, USPS_dir=project_folder + 'USPS/')

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                         validation_size=len(test_ds.images))
    test_set = tf.data.Dataset.from_tensor_slices((test_ds.images, test_ds.labels)).batch(bs)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")

        for i in range(trials):

            if history[j, i, k, 0] != 0:
                continue

            print(f"Trial {i + 1}")

            model = create_model(num_units=2 * trees)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).batch(bs)

            with tf.device('/device:GPU:0'):

                fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                        validation_data=valid_set, validation_batch_size=bs,
                                        callbacks=[callback], verbose=0)
                print_fit_history(fit_history, epochs)

                evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
                print_evaluate_history(evaluate_history)

                history[j, i, k] = evaluate_history[1:]

                np.save(project_folder + 'results/fcnn_history.npy', history,
                        allow_pickle=True)

Dataset: usps / Pair: 3-5
1-FCNN
Trial 1
Epochs: 622/2000 - Train BCE: 0.0282, accuracy: 100.0% - Validation BCE: 0.0918, accuracy: 96.97%
Test BCE: 0.2943, accuracy: 88.34%
Trial 2
Epochs: 2000/2000 - Train BCE: 0.0739, accuracy: 98.88% - Validation BCE: 0.0498, accuracy: 100.0%
Test BCE: 0.2754, accuracy: 90.8%
Trial 3
Epochs: 711/2000 - Train BCE: 0.0153, accuracy: 100.0% - Validation BCE: 0.1779, accuracy: 96.97%
Test BCE: 0.3047, accuracy: 88.96%
Trial 4
Epochs: 607/2000 - Train BCE: 0.0236, accuracy: 100.0% - Validation BCE: 0.1144, accuracy: 90.91%
Test BCE: 0.2245, accuracy: 92.02%
Trial 5
Epochs: 2000/2000 - Train BCE: 0.0211, accuracy: 100.0% - Validation BCE: 0.0569, accuracy: 96.97%
Test BCE: 0.3154, accuracy: 88.96%
Trial 6
Epochs: 648/2000 - Train BCE: 0.1797, accuracy: 100.0% - Validation BCE: 0.2485, accuracy: 90.91%
Test BCE: 0.3865, accuracy: 86.81%
Trial 7
Epochs: 2000/2000 - Train BCE: 0.0937, accuracy: 100.0% - Validation BCE: 0.214, accuracy: 93.75%
Test BCE: 0.33

KeyboardInterrupt: 

In [ ]:
history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{k}-FCNN")
        print(f"Accuracy: mean = {round(np.mean(history[j,:,k,1]), 4)}, standard deviation = {round(np.std(history[j,:,k,1]), 4)}")